In [1]:
import cv2
import numpy as np

In [31]:
image = cv2.imread(r'S:/IMAGES/WARANGAL/DSC_4417.jpg')
cv2.imshow('original_image',image)

cv2.waitKey(0)
cv2.destroyWindow('original_image')

w,h = image.shape[:2]

In [10]:
blob = cv2.dnn.blobFromImage(image,1./255,(416,416),swapRB = True,crop = False)

print(image.shape)
print(blob.shape)
blob_to_show = blob[0,:,:,:].transpose(1,2,0)

print(blob_to_show.shape)

cv2.namedWindow('blob to show',cv2.WINDOW_NORMAL)
cv2.imshow('blob to show',cv2.cvtColor(blob_to_show,cv2.COLOR_BGR2RGB))

cv2.waitKey(0)
cv2.destroyAllWindows()

(3000, 4496, 3)
(1, 3, 416, 416)
(416, 416, 3)


In [12]:
with open(r'coco.names') as f:
    labels = [line.strip() for line in f]

print(labels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [13]:
network = cv2.dnn.readNetFromDarknet('yolov3.cfg','yolov3.weights')

In [23]:
layers_all = network.getLayerNames()

output_layers = [layers_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

print(output_layers)

threshould = 0.7
probability = 0.6

color = np.random.randint(0,255,size = (len(labels),3), dtype= 'int')

['yolo_82', 'yolo_94', 'yolo_106']


In [25]:
print(color[1])

[176 155 241]


In [26]:
network.setInput(blob)
output = network.forward(output_layers)
print(output)

[array([[0.03390884, 0.04577182, 0.5032545 , ..., 0.        , 0.        ,
        0.        ],
       [0.04396925, 0.03170494, 0.31457514, ..., 0.        , 0.        ,
        0.        ],
       [0.04481798, 0.03409215, 0.80637383, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.95759195, 0.95279384, 0.3389671 , ..., 0.        , 0.        ,
        0.        ],
       [0.96100205, 0.95830345, 0.25674224, ..., 0.        , 0.        ,
        0.        ],
       [0.9696175 , 0.9636785 , 0.7716408 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.0178851 , 0.0263138 , 0.0524288 , ..., 0.        , 0.        ,
        0.        ],
       [0.01918899, 0.02361843, 0.33401898, ..., 0.        , 0.        ,
        0.        ],
       [0.02221562, 0.01763069, 0.08022236, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.97651714, 0.97480625, 0.04373187, ..., 0.        , 0.        ,
        0.        ],
       [0.9778502 

In [39]:
bounding_boxes = []
probabilities = []
class_numbers = []

for i in output:
    for detected_object in i:
        scores = detected_object[:5]
        indicies_of_maxscore = np.argmax(scores)
        class_scores = scores[indicies_of_maxscore]
        
        
        if class_scores>probability:
            box_current = detected_object[0:4] * ([w,h,w,h])
            x_center,y_center,height,width = box_current
            x_min = (x_center - (width/2))
            y_min = (y_center - (height/2))
            
            bounding_boxes.append([x_min,y_min,int(width),int(height)])
            probabilities.append(float(class_scores))
            class_numbers.append(indicies_of_maxscore)

In [42]:
nmsboxes = cv2.dnn.NMSBoxes(bounding_boxes,probabilities,probability,threshould)